# Exercise 1: Introduksjon til GenAI med Microsoft Fluent UI Emojis

## Del 1: Forberedelse av Microsoft Fluent UI Emojis datasettetI denne oppgaven skal vi bruke Microsoft Fluent UI Emojis datasettet for å trene en klassifikasjonsmodell. Dette datasettet inneholder moderne emoji-ikoner i forskjellige stiler.

In [ ]:
# Nødvendige importsimport warningsimport osimport mathimport randomimport numpy as npimport torchimport torch.nn as nnimport torch.optim as optimimport matplotlib.pyplot as pltimport requestsimport zipfileimport iofrom PIL import Imagefrom torchvision import transforms, datasetsfrom torch.utils.data import DataLoader, Dataset, random_split# Undertrykk advarsler for renere outputwarnings.filterwarnings("ignore")# Sikre reproduserbarhetSEED = 42random.seed(SEED)np.random.seed(SEED)torch.manual_seed(SEED)torch.cuda.manual_seed_all(SEED)torch.backends.cudnn.deterministic = Truetorch.backends.cudnn.benchmark = False# Sjekk for GPU-akselerasjondevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")print("Enhet:", device)

## Last ned og forbered Microsoft Fluent UI Emojis datasettetVi skal laste ned Microsoft Fluent UI Emojis datasettet fra GitHub og forberede det for trening.

In [ ]:
# Definer stien til datasettetdataset_path = os.path.join(os.getcwd(), 'fluent_emojis')os.makedirs(dataset_path, exist_ok=True)print(f"Dataset vil bli lagret i: {dataset_path}")# Last ned Microsoft Fluent UI Emojis datasettetdef download_fluent_emojis():    # URL til Microsoft Fluent UI Emojis GitHub repository    url = 'https://github.com/microsoft/fluentui-emoji/archive/refs/heads/main.zip'        print("Laster ned Microsoft Fluent UI Emojis datasettet...")    response = requests.get(url)        if response.status_code == 200:        print("Nedlasting fullført. Pakker ut filer...")        z = zipfile.ZipFile(io.BytesIO(response.content))        z.extractall(dataset_path)        print("Utpakking fullført.")        return os.path.join(dataset_path, 'fluentui-emoji-main')    else:        print(f"Feil ved nedlasting: {response.status_code}")        return None# Last ned datasettetemoji_path = download_fluent_emojis()

## Utforsk datasettetLa oss utforske strukturen til datasettet og se på noen eksempler.

In [ ]:
def explore_dataset(emoji_path):    if emoji_path is None or not os.path.exists(emoji_path):        print("Datasettet er ikke tilgjengelig.")        return        # Finn alle kategorimapper    categories = [d for d in os.listdir(emoji_path) if os.path.isdir(os.path.join(emoji_path, d)) and not d.startswith('.')]    print(f"Antall emoji-kategorier: {len(categories)}")    print(f"Eksempler på kategorier: {categories[:10]}")        # Vis noen eksempler på emojis    plt.figure(figsize=(15, 10))    sample_count = min(5, len(categories))    for i, category in enumerate(categories[:sample_count]):        category_path = os.path.join(emoji_path, category)        style_dirs = [d for d in os.listdir(category_path) if os.path.isdir(os.path.join(category_path, d))]                for j, style in enumerate(style_dirs[:2]):  # Vis 2 stiler per kategori            style_path = os.path.join(category_path, style)            png_files = [f for f in os.listdir(style_path) if f.endswith('.png')]                        if png_files:                img_path = os.path.join(style_path, png_files[0])                img = Image.open(img_path)                plt.subplot(sample_count, 2, i*2 + j + 1)                plt.imshow(img)                plt.title(f"{category} - {style}")                plt.axis('off')        plt.tight_layout()    plt.show()        return categories# Utforsk datasettetcategories = explore_dataset(emoji_path)

## Forbered datasettet for treningVi skal nå forberede datasettet for trening ved å lage en tilpasset PyTorch Dataset-klasse.

In [ ]:
class FluentEmojiDataset(Dataset):    def __init__(self, emoji_path, categories=None, transform=None, max_per_category=50):        self.transform = transform        self.samples = []        self.class_to_idx = {}                if categories is None or len(categories) == 0:            return                # Begrens til et håndterbart antall kategorier for trening        selected_categories = categories[:20]  # Bruk de første 20 kategoriene                for idx, category in enumerate(selected_categories):            self.class_to_idx[category] = idx            category_path = os.path.join(emoji_path, category)            style_dirs = [d for d in os.listdir(category_path) if os.path.isdir(os.path.join(category_path, d))]                        count = 0            for style in style_dirs:                style_path = os.path.join(category_path, style)                png_files = [f for f in os.listdir(style_path) if f.endswith('.png')]                                for png_file in png_files:                    if count >= max_per_category:                        break                    img_path = os.path.join(style_path, png_file)                    self.samples.append((img_path, idx))                    count += 1                                if count >= max_per_category:                    break                print(f"Totalt antall bilder: {len(self.samples)}")        print(f"Antall klasser: {len(self.class_to_idx)}")        def __len__(self):        return len(self.samples)        def __getitem__(self, idx):        img_path, label = self.samples[idx]        image = Image.open(img_path).convert('RGB')                if self.transform:            image = self.transform(image)                return image, label# Definer transformasjonertransform = transforms.Compose([    transforms.Resize((64, 64)),    transforms.ToTensor(),    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])# Opprett datasettetdataset = FluentEmojiDataset(emoji_path, categories=categories, transform=transform)# Del datasettet i trening og valideringtrain_size = int(0.8 * len(dataset))val_size = len(dataset) - train_sizetrain_dataset, val_dataset = random_split(dataset, [train_size, val_size])# Opprett dataloadersbatch_size = 32train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)val_loader = DataLoader(val_dataset, batch_size=batch_size)print(f"Treningssett størrelse: {len(train_dataset)}")print(f"Valideringssett størrelse: {len(val_dataset)}")

## Definer CNN-modellenVi skal nå definere en enkel CNN-modell for å klassifisere emojiene.

In [ ]:
class EmojiCNN(nn.Module):    def __init__(self, num_classes):        super(EmojiCNN, self).__init__()        self.features = nn.Sequential(            nn.Conv2d(3, 32, kernel_size=3, padding=1),            nn.ReLU(inplace=True),            nn.MaxPool2d(kernel_size=2, stride=2),                        nn.Conv2d(32, 64, kernel_size=3, padding=1),            nn.ReLU(inplace=True),            nn.MaxPool2d(kernel_size=2, stride=2),                        nn.Conv2d(64, 128, kernel_size=3, padding=1),            nn.ReLU(inplace=True),            nn.MaxPool2d(kernel_size=2, stride=2),        )                self.classifier = nn.Sequential(            nn.Linear(128 * 8 * 8, 512),            nn.ReLU(inplace=True),            nn.Dropout(0.5),            nn.Linear(512, num_classes)        )        def forward(self, x):        x = self.features(x)        x = x.view(x.size(0), -1)        x = self.classifier(x)        return x# Opprett modellennum_classes = len(dataset.class_to_idx)model = EmojiCNN(num_classes).to(device)print(model)

## Tren modellenNå skal vi trene modellen på datasettet.

In [ ]:
# Definer tap og optimaliserercriterion = nn.CrossEntropyLoss()optimizer = optim.Adam(model.parameters(), lr=0.001)# Treningsfunksjondef train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):    train_losses = []    val_losses = []    train_accuracies = []    val_accuracies = []        for epoch in range(num_epochs):        # Trening        model.train()        running_loss = 0.0        correct = 0        total = 0                for inputs, labels in train_loader:            inputs, labels = inputs.to(device), labels.to(device)                        optimizer.zero_grad()            outputs = model(inputs)            loss = criterion(outputs, labels)            loss.backward()            optimizer.step()                        running_loss += loss.item() * inputs.size(0)            _, predicted = torch.max(outputs, 1)            total += labels.size(0)            correct += (predicted == labels).sum().item()                epoch_loss = running_loss / len(train_loader.dataset)        epoch_acc = correct / total        train_losses.append(epoch_loss)        train_accuracies.append(epoch_acc)                # Validering        model.eval()        val_loss = 0.0        val_correct = 0        val_total = 0                with torch.no_grad():            for inputs, labels in val_loader:                inputs, labels = inputs.to(device), labels.to(device)                outputs = model(inputs)                loss = criterion(outputs, labels)                                val_loss += loss.item() * inputs.size(0)                _, predicted = torch.max(outputs, 1)                val_total += labels.size(0)                val_correct += (predicted == labels).sum().item()                val_epoch_loss = val_loss / len(val_loader.dataset)        val_epoch_acc = val_correct / val_total        val_losses.append(val_epoch_loss)        val_accuracies.append(val_epoch_acc)                print(f"Epoch {epoch+1}/{num_epochs} => "              f"Trening: Tap: {epoch_loss:.4f}, Nøyaktighet: {epoch_acc:.4f} | "              f"Validering: Tap: {val_epoch_loss:.4f}, Nøyaktighet: {val_epoch_acc:.4f}")        return train_losses, val_losses, train_accuracies, val_accuracies# Tren modellennum_epochs = 10train_losses, val_losses, train_accuracies, val_accuracies = train_model(    model, train_loader, val_loader, criterion, optimizer, num_epochs)

## Visualiser treningsresultaterLa oss visualisere treningsresultatene.

In [ ]:
# Plot tap og nøyaktighetplt.figure(figsize=(12, 5))plt.subplot(1, 2, 1)plt.plot(train_losses, label='Trening')plt.plot(val_losses, label='Validering')plt.title('Tap over epoker')plt.xlabel('Epoke')plt.ylabel('Tap')plt.legend()plt.subplot(1, 2, 2)plt.plot(train_accuracies, label='Trening')plt.plot(val_accuracies, label='Validering')plt.title('Nøyaktighet over epoker')plt.xlabel('Epoke')plt.ylabel('Nøyaktighet')plt.legend()plt.tight_layout()plt.show()

## Test modellen på noen eksemplerLa oss teste modellen på noen eksempler fra valideringssettet.

In [ ]:
# Få klasse-navn fra indeksidx_to_class = {v: k for k, v in dataset.class_to_idx.items()}# Funksjon for å vise prediksjonerdef show_predictions(model, val_loader, idx_to_class, num_samples=5):    model.eval()    images, labels = next(iter(val_loader))    images, labels = images[:num_samples].to(device), labels[:num_samples].to(device)        with torch.no_grad():        outputs = model(images)        _, preds = torch.max(outputs, 1)        # Konverter bilder tilbake til visningsformat    images = images.cpu().numpy()    mean = np.array([0.485, 0.456, 0.406])    std = np.array([0.229, 0.224, 0.225])        # Denormalisering    images = std * images.transpose(0, 2, 3, 1) + mean    images = np.clip(images, 0, 1)        # Vis bilder med prediksjoner    plt.figure(figsize=(15, 3))    for i in range(num_samples):        plt.subplot(1, num_samples, i + 1)        plt.imshow(images[i])        true_label = idx_to_class[labels[i].item()]        pred_label = idx_to_class[preds[i].item()]        title = f"Sann: {true_label}
Pred: {pred_label}"        plt.title(title)        plt.axis('off')        plt.tight_layout()    plt.show()# Vis noen prediksjonershow_predictions(model, val_loader, idx_to_class)

## Lagre modellenLa oss lagre den trente modellen for senere bruk.

In [ ]:
# Lagre modellenmodel_path = os.path.join(os.getcwd(), 'emoji_classifier_model.pth')torch.save({    'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),    'class_to_idx': dataset.class_to_idx,    'num_classes': num_classes}, model_path)print(f"Modell lagret til {model_path}")

## KonklusjonI denne oppgaven har vi:1. Lastet ned og utforsket Microsoft Fluent UI Emojis datasettet2. Forberedt datasettet for trening ved å lage en tilpasset PyTorch Dataset-klasse3. Definert en CNN-modell for å klassifisere emojiene4. Trent modellen på datasettet5. Visualisert treningsresultatene6. Testet modellen på noen eksempler7. Lagret modellen for senere brukDette er en grunnleggende implementasjon som kan utvides på flere måter:- Bruke mer avanserte CNN-arkitekturer som ResNet eller EfficientNet- Implementere dataaugmentering for å forbedre modellens robusthet- Bruke transfer learning fra pre-trente modeller- Eksperimentere med forskjellige hyperparametere